(Question 1)

In [1]:
from bs4 import BeautifulSoup
from tabulate import tabulate
import pandas as pd
import numpy as np

In [2]:
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

In [3]:
import requests

In [4]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')

In [5]:
tb = soup.find('table', class_ = 'wikitable')

In [6]:
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))


In [8]:
postcodes = df[0]

In [9]:
#Remove 'Not assigned' boroughs
postcodes_clean = postcodes[postcodes['Borough']!='Not assigned']

In [10]:
#Update new panda dataframe by grouping the neighbourhoods and boroughs by postcode.
#postcodes_clean = postcodes_clean.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(', '.join).reset_index()

In [11]:
postcodes_clean = postcodes_clean.groupby(['Postcode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()

In [12]:
for i in range(len(postcodes_clean)):
    if postcodes_clean['Neighbourhood'][i] == 'Not assigned':
        postcodes_clean['Neighbourhood'][i] = postcodes_clean['Borough'][i]

## Question 2

In [14]:
cordinates = pd.read_csv("Geospatial_Coordinates.csv")

In [15]:
cordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [16]:
cordinates.rename(columns={'Postal Code':'Postcode'}, 
                 inplace=True)

In [17]:
df = pd.merge(postcodes_clean, cordinates, on="Postcode")
df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
